# Connect to Cohere Command-R+ LLM

"Command R+, like our recently launched Command R model, features a 128k-token context window and is designed to offer best-in-class:

Advanced Retrieval Augmented Generation (RAG) with citation to reduce hallucinations
Multilingual coverage in 10 key languages to support global business operations
Tool Use to automate sophisticated business processes". Source: [txt.cohere.com]("https://txt.cohere.com/command-r-plus-microsoft-azure/")

In [23]:
import dspy
command_r_plus = dspy.Cohere(model="command-r-plus", max_tokens=4000, api_key=cohere_api_key)

command_r_plus("""Answer the question based on the following context:

Features released in Weaviate 1.24
<li>Named vectors. A single object can have multiple vectors. Create vectors for properties, use different vectorization models, and apply different metrics to fine tune interactions with your data.
<li>HNSW and binary quantization (BQ) HNSW indexes and BQ combine for serious compression and blazing speed.
<li>Simplified Docker configuration. A new Docker image that needs no configuration.
<li>Backend improvements. Numerous improvements to make updates, imports, and deletions faster.
<li>Python client update. General availability, support for all the latest features.

question: Can I use Binary Quantization in Weaviate?""")

['Yes, Binary Quantization (BQ) is available in Weaviate 1.24 and can be used in combination with HNSW indexes for improved compression and speed.']

# Connect to Weaviate

### Weaviate Blogs

This weaviate instance contains the 80 blog posts from the Weaviate team, totaling 1,200 text chunks. This covers diverse information about Weaviate from release notes to case studies, research, and more!

In [24]:
import weaviate
from dspy.retrieve.weaviate_rm import WeaviateRM
weaviate_client = weaviate.connect_to_local()
retriever_model = WeaviateRM("WeaviateBlogChunk", weaviate_client=weaviate_client, k=10)

data = [d["long_text"] for d in retriever_model("What is ref2vec?")]
formatted_string = "\n".join(f"[{i+1}] {text}" for i, text in enumerate(data))
print(formatted_string)

[1] ---
title: What is Ref2Vec and why you need it for your recommendation system
slug: ref2vec-centroid
authors: [connor]
date: 2022-11-23
tags: ['integrations', 'concepts']
image: ./img/hero.png
description: "Weaviate introduces Ref2Vec, a new module that utilises Cross-References for Recommendation!"
---
![Ref2vec-centroid](./img/hero.png)

<!-- truncate -->

Weaviate 1.16 introduced the [Ref2Vec](/developers/weaviate/modules/retriever-vectorizer-modules/ref2vec-centroid) module. In this article, we give you an overview of what Ref2Vec is and some examples in which it can add value such as recommendations or representing long objects. ## What is Ref2Vec? The name Ref2Vec is short for reference-to-vector, and it offers the ability to vectorize a data object with its cross-references to other objects. The Ref2Vec module currently holds the name ref2vec-**centroid** because it uses the average, or centroid vector, of the cross-referenced vectors to represent the **referencing** object.

/var/folders/41/8dp_379x15d8zz4ppsjthdw40000gn/T/ipykernel_31325/3014742529.py:4: ResourceWarning: unclosed <socket.socket fd=87, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=6, laddr=('::1', 57287, 0, 0), raddr=('::1', 8080, 0, 0)>
  retriever_model = WeaviateRM("WeaviateBlogChunk", weaviate_client=weaviate_client, k=10)


### Long Context RAG

Because Command-R-Plus supports a 128K input window, we can test much higher values of `k` for retrieval than previous generation RAG systems.

For example, the `k=10` search results above contain about `1,900 tokens`.

# Set DSPy default LLM to Command R Plus and RM to Weaviate

In [25]:
dspy.settings.configure(lm=command_r_plus, rm=retriever_model)

# Define RAG Progarm

In [26]:
class GenerateAnswer(dspy.Signature):
    """Assess the the context and answer the question."""

    context = dspy.InputField(desc="Helpful information for answering the question.")
    question = dspy.InputField()
    detailed_answer = dspy.OutputField(desc="A detailed answer that is supported by the context. Only output the answer.")
    
class RAG(dspy.Module):
    def __init__(self, k=10):
        super().__init__()
        
        self.retrieve = dspy.Retrieve(k=k)
        self.generate_answer = dspy.Predict(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        pred = self.generate_answer(context=context, question=question).detailed_answer
        return dspy.Prediction(context=context, answer=pred, question=question)

## Query Time

In [27]:
RAG()(question="How does HNSW vector search work?").answer

'HNSW, or Hierarchical Navigable Small World graphs, is an ANN (Approximate Nearest Neighbor) algorithm that powers Weaviate, a vector database. HNSW organizes vector indexes so that closely related vectors are stored next to each other, enabling fast and efficient vector searches. By trading a small amount of accuracy for significant performance gains, HNSW can maintain high recall rates (>95%) while achieving high throughput and low latency, making it ideal for fast and reliable vector searches.'